## GuardSQL

In [57]:
import json

import openai
import pandas as pd

import guardrails as gd

from utils import sqllite_db_to_prompt

In [58]:
rail_spec = """
<rail version="0.1">

<output>
    <sql
        name="generated_sql"
        description="Generate SQL for the given natural language instruction."
        format="bug-free-sql"
        on-fail-bug-free-sql="reask" 
    />
</output>


<prompt>

Generate a valid SQL query for the following natural language instruction:

{{nl_instruction}}

Here's schema about the database that you can use to generate the SQL query.
Try to avoid using joins if the data can be retrieved from the same table.

{{db_schema}}

@complete_json_suffix
</prompt>


</rail>
"""

In [59]:
guard = gd.Guard.from_rail_string(rail_spec)

In [41]:
dataset = []

with open('/Users/shreyarajpal/Downloads/spider/train_spider.json') as f:
    data = json.load(f)
    for row in data:
        dataset.append({
            'db_id': row['db_id'],
            'query': row['query'],
            'question': row['question']}
        )

df = pd.DataFrame(dataset)

In [42]:
df

,db_id,query,question
0,department_management,SELECT count(*) FROM head WHERE age > 56,How many heads of the departments are older th...
1,department_management,"SELECT name , born_state , age FROM head ORD...","List the name, born state and age of the heads..."
2,department_management,"SELECT creation , name , budget_in_billions ...","List the creation year, name and budget of eac..."
3,department_management,"SELECT max(budget_in_billions) , min(budget_i...",What are the maximum and minimum budget of the...
4,department_management,SELECT avg(num_employees) FROM department WHER...,What is the average number of employees of the...
...,...,...,...
6995,culture_company,SELECT T1.company_name FROM culture_company AS...,What are all the company names that have a boo...
6996,culture_company,"SELECT T1.title , T3.book_title FROM movie AS...",Show the movie titles and book titles for all ...
6997,culture_company,"SELECT T1.title , T3.book_title FROM movie AS...",What are the titles of movies and books corres...
6998,culture_company,SELECT T2.company_name FROM movie AS T1 JOIN c...,Show all company names with a movie directed i...


In [43]:
def nl2sql(nl_instruction: str, db_id: str) -> str:

    db_info = sqllite_db_to_prompt(f'/Users/shreyarajpal/Downloads/spider/database/{db_id}/{db_id}.sqlite')

    with open(f'/Users/shreyarajpal/Downloads/spider/database/{db_id}/schema.sql') as f:
        db_schema = f.read()

    response = guard(
        openai.Completion.create,
        prompt_params={
            'nl_instruction': nl_instruction,
            # 'db_info': db_info,
            'db_schema': db_schema,
        },
        engine='text-davinci-003',
        temperature=0.0,
        max_tokens=512,
    )
    return response[1]['generated_sql']

In [51]:
df_sample = df[:10]
df_sample

,db_id,query,question
0,department_management,SELECT count(*) FROM head WHERE age > 56,How many heads of the departments are older th...
1,department_management,"SELECT name , born_state , age FROM head ORD...","List the name, born state and age of the heads..."
2,department_management,"SELECT creation , name , budget_in_billions ...","List the creation year, name and budget of eac..."
3,department_management,"SELECT max(budget_in_billions) , min(budget_i...",What are the maximum and minimum budget of the...
4,department_management,SELECT avg(num_employees) FROM department WHER...,What is the average number of employees of the...
5,department_management,SELECT name FROM head WHERE born_state != 'Cal...,What are the names of the heads who are born o...
6,department_management,SELECT DISTINCT T1.creation FROM department AS...,What are the distinct creation years of the de...
7,department_management,SELECT born_state FROM head GROUP BY born_stat...,What are the names of the states where at leas...
8,department_management,SELECT creation FROM department GROUP BY creat...,In which year were most departments established?
9,department_management,"SELECT T1.name , T1.num_employees FROM depart...",Show the name and number of employees for the ...


In [52]:
# Apply a function to each row of the dataframe for the first 10 rows
# Add the output of this function as  a new column in the dataframe

df_sample['generated_sql'] = df_sample.apply(lambda row: nl2sql(row['question'], row['db_id']), axis=1)

# output = df[:1].apply(lambda row: nl2sql(row['question'], row['db_id']), axis=1)

/Users/shreyarajpal/anaconda3/envs/tiff-env/lib/python3.9/site-packages/eliot/json.py:22: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.  (This may have returned Python scalars in past versions.
  if isinstance(o, (numpy.bool, numpy.bool_)):
/var/folders/rw/j_7mc6vs7ps0f9kl1mrgb6k40000gn/T/ipykernel_7804/985577230.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample['generated_sql'] = df_sample.apply(lambda row: nl2sql(row['question'], row['db_id']), axis=1)


In [55]:
# Display only the columns question and generated_sql for the first 10 rows

df_sample[['question', 'generated_sql', 'query']]

,question,generated_sql,query
0,How many heads of the departments are older th...,SELECT COUNT(*) FROM head WHERE age > 56,SELECT count(*) FROM head WHERE age > 56
1,"List the name, born state and age of the heads...","SELECT name, born_state, age FROM head ORDER B...","SELECT name , born_state , age FROM head ORD..."
2,"List the creation year, name and budget of eac...","SELECT Creation, Name, Budget_in_Billions FROM...","SELECT creation , name , budget_in_billions ..."
3,What are the maximum and minimum budget of the...,"SELECT MAX(Budget_in_Billions), MIN(Budget_in_...","SELECT max(budget_in_billions) , min(budget_i..."
4,What is the average number of employees of the...,SELECT AVG(Num_Employees) FROM department WHER...,SELECT avg(num_employees) FROM department WHER...
5,What are the names of the heads who are born o...,SELECT name FROM head WHERE born_state != 'Cal...,SELECT name FROM head WHERE born_state != 'Cal...
6,What are the distinct creation years of the de...,SELECT DISTINCT Creation FROM department INNER...,SELECT DISTINCT T1.creation FROM department AS...
7,What are the names of the states where at leas...,SELECT born_state FROM head GROUP BY born_stat...,SELECT born_state FROM head GROUP BY born_stat...
8,In which year were most departments established?,"SELECT Creation, COUNT(*) AS Num_Departments F...",SELECT creation FROM department GROUP BY creat...
9,Show the name and number of employees for the ...,"SELECT Name, Num_Employees FROM department d I...","SELECT T1.name , T1.num_employees FROM depart..."
